# Continuous Integration using Jenkins

### Ngrok

Go to https://ngrok.com and click on SignUp.  You can create an account using your GitHub login.

Navigate to https://dashboard.ngrok.com/get-started/setup and add the auth token to a `.env` file:

Add a `.gitignore` file that ignores `.env` files:

### Building a local Jenkins environment

Using the installation instructions for Docker at https://jenkins.io/doc/book/installing/, create a local Docker Compose environment:

Create a Dockerfile called `Dockerfile.ngrok` that installs ngrok to allow the local Jenkins container to be externally accessible:

Create a Dockerfile called `Dockerfile.agent` that will provide a Docker environment to run Jenkins builds.

The agent only requires make, docker client and docker compose.

Create a Dockerfile called `Dockerfile.agent` that will provide a Docker environment to run Jenkins builds.

The agent only requires make, docker client and docker compose.

Create a simple Makefile to automate various tasks for building and starting the Jenkins environment:

Build and publish the agent image by running `make agent`:

In [ ]:
$ make agent
...
...

Start up environment by running `make jenkins`.  The output should show initial admin password:

Browse to your configured ngrok url (e.g. https://cd-docker-jenkins.ngrok.io) or alternatively http://localhost:8080 and enter the initial admin password.

When asked to install plugins, choose Install Suggested Plugins option.

Add a new user called admin when prompted.

In Instance Configuration, ensure the URL is configured as your external ngrok URL (e.g. https://cd-docker-jenkins.ngrok.io/)

When asked to install plugins, choose customise option and ensure the following plugins are selected:

- GitHub
- Pipeline GitHub Notify Step Plugin
- Pipeline Utility Steps
- disable-github-multibranch-status-plugin

### Configure GitHub Plugin

In GitHub create a new personal token with the following scopes:

- repo
- admin:repo_hook
- read:user
- read:email

Browse to **Manage Jenkins -> System**.

In GitHub section add a new GitHub server called GitHub, adding a new credential of type secret text with the GitHub personal token and enabling Manage Hooks.  Click on Test Connection to verify the configuration and then Save.

### Create a new pipeline

Use BlueOcean wizard to create an initial pipeline that runs:

- make build
- make test
- make release
- make clean

Configure the pipeline to use Docker agent image.

### Configure pipeline to collect test reports

In post section of Jenkinsfile, add junit step:

### Configure pipeline to publish Docker images

Browse to **Credentials -> Global** and add two new credentials:

- docker-user (secret text) - set the username of Docker Hub account
- docker-password (secret file) - upload a file with the password of Docker Hub account (this approach is more secure)

Add login and logout tasks to Makefile:

Add environment variables to Jenkinsfile:

Add publish stage to Jenkinsfile (after release stage) and add `make logout` task to post:

The `make login` command because the job runs as user ID 1000 in the Docker agent.

To resolve this add a tmpfs mount in the docker agent configuration:

### Configure pipeline for a PR-based CD workflow

Configure the pipeline job in Jenkins as follows:

1. Add **Filter by name (with wildcards)** behaviour to branch sources
2. Configure the include filter with the expression `master PR*`
3. Add **Disable GitHub Notifications** behaviour to branch sources

This configuration will only trigger the pipeline for changes to the master branch or for branches that have an open pull request, and will disable the default GitHub notification behaviour.

Create a new branch called `pr_workflow` or similar and configure the build, test, release and publish stages to only execute when the pipeline is triggered by a pull request:

Commit the change and push the branch to GitHub - at this point because the branch is not the master branch and no PR has been created, no build should be triggered in Jenkins.

Next, create a pull request.  Now a build should be triggered for the pull request.  Notice that Jenkins publishes a status check into GitHub indicating the build is in progress, and eventually that the build succeeded or failed.

If you now merge the PR, notice that when the master build is triggered, the build no longer does anything.

### Configure a Branch Protection Rule in GitHub

Configure a branch protection rule for the master branch that includes the following settings:
    
- Enable **Require status checks to pass before merging**
- Enable **Require branches to be up to date before merging**
- Enable **continuous-integration/jenkins/pr-merge** as a required status check
- Enable **Include administrators**

After saving this branch protection rule, you should no longer be able to push directly to the master branch

In **Settings -> Options** ensure only the **Allow merge commits** setting is enabled in Merge Button section

### Creating Releases on Merge to Master

Create a new branch called `publish_and_tag_releases` or similar.

In the `Makefile` add the following variables.

The GIT_TAG variable prints latest semantic version tag
The PR_COMMIT variable shows the 2nd most recent parent as related to HEAD (1st parent commit is the master branch merge commit), which is the PR commit before it was merged.  This is important as this is what our built Docker image was tagged with.

Create a file called `VERSION` and add an initial target semantic version:

Add the following to the bottom of the Makefile.

This will compare the current git tag with the target version in the `VERSION` file, and take the higher version of the following:

- Bumped git tag version (e.g. if current git tag is 0.4.3, the bumped version is 0.4.4)
- Target version

E.g. if target version is `0.1.0` then the bumped git tag version of `0.4.4` becomes the new version, if target version is `0.5.0` then `0.5.0` becomes the new version.

Add a `version` task to the Makefile:

Add a `tag` task to the Makefile.

This task first pulls the PR_COMMIT image (remember the PR must build and publish this image successfully for the PR to be merged), and then tags/pushes the image with the next version and `latest` tags.  Finally a new tag and release is created in GitHub using the `hub` utility.

Add a `tag` stage to the Jenkinsfile - also need to add `GITHUB_TOKEN` to the environment section to ensure the `hub` command in the `tag` task works.

The `tag` stage must be configured to only run for the master branch:

Merge the Pull Request.  In Jenkins the master job should now execute the `tag` stage, which results in tagged Docker image and a new GitHub release.